<a href="https://colab.research.google.com/github/PranavkrishnaVadhyar/AI_First_Aid/blob/main/Wound_Classification_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

In [2]:
!unzip "/content/drive/MyDrive/archive (2).zip" -d "/content/data/"

Archive:  /content/drive/MyDrive/archive (2).zip
  inflating: /content/data/Wound_dataset/Abrasions/abrasions (1).jpg  
  inflating: /content/data/Wound_dataset/Abrasions/abrasions (10).jpg  
  inflating: /content/data/Wound_dataset/Abrasions/abrasions (11).jpg  
  inflating: /content/data/Wound_dataset/Abrasions/abrasions (12).jpg  
  inflating: /content/data/Wound_dataset/Abrasions/abrasions (13).jpg  
  inflating: /content/data/Wound_dataset/Abrasions/abrasions (14).jpg  
  inflating: /content/data/Wound_dataset/Abrasions/abrasions (15).jpg  
  inflating: /content/data/Wound_dataset/Abrasions/abrasions (16).jpg  
  inflating: /content/data/Wound_dataset/Abrasions/abrasions (17).jpg  
  inflating: /content/data/Wound_dataset/Abrasions/abrasions (18).jpg  
  inflating: /content/data/Wound_dataset/Abrasions/abrasions (19).jpg  
  inflating: /content/data/Wound_dataset/Abrasions/abrasions (2).jpg  
  inflating: /content/data/Wound_dataset/Abrasions/abrasions (20).jpg  
  inflating: /con

In [3]:
import os
import shutil

# Define the path to your dataset directory
dataset_directory = "/content/data/Wound_dataset"

# Define the path to your output directory where the organized dataset will be stored
output_directory = "/content/data/Wounds"

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# List all subdirectories in the dataset directory (each subdirectory represents a class)
class_directories = [d for d in os.listdir(dataset_directory) if os.path.isdir(os.path.join(dataset_directory, d))]

# Iterate through each class directory
for class_dir in class_directories:
    # Create a directory with the same class name in the output directory
    output_class_dir = os.path.join(output_directory, class_dir)
    os.makedirs(output_class_dir, exist_ok=True)

    # List all files in the class directory (these are the images)
    image_files = [f for f in os.listdir(os.path.join(dataset_directory, class_dir)) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Copy each image to the corresponding class directory in the output directory
    for image_file in image_files:
        src_path = os.path.join(dataset_directory, class_dir, image_file)
        dst_path = os.path.join(output_class_dir, image_file)
        shutil.copy(src_path, dst_path)

print("Dataset preparation completed.")


Dataset preparation completed.


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define dataset directory and parameters
dataset_directory = '/content/data/Wounds'
batch_size = 32
image_size = (224, 224)
num_classes = len(os.listdir(dataset_directory))

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generator for training set
train_generator = train_datagen.flow_from_directory(
    dataset_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multi-class classification
    shuffle=True
)

# Data generator for testing set (no augmentation)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    dataset_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multi-class classification
    shuffle=False
)


Found 431 images belonging to 7 classes.
Found 431 images belonging to 7 classes.


#ResNet50

In [5]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)  # Add more layers if needed
output = Dense(num_classes, activation='softmax')(x)  # Adjust for the number of classes

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


94765736/94765736 [==============================] - 0s 0us/step


In [6]:
epochs = 10  # Adjust the number of training epochs

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator
)


Epoch 1/10
14/14 [==============================] - 23s 716ms/step - loss: 2.0871 - accuracy: 0.2251 - val_loss: 1.9266 - val_accuracy: 0.1415
Epoch 2/10
14/14 [==============================] - 9s 664ms/step - loss: 1.8727 - accuracy: 0.2343 - val_loss: 1.8828 - val_accuracy: 0.2993
Epoch 3/10
14/14 [==============================] - 7s 538ms/step - loss: 1.8369 - accuracy: 0.2599 - val_loss: 1.8777 - val_accuracy: 0.1323
Epoch 4/10
14/14 [==============================] - 8s 554ms/step - loss: 1.8849 - accuracy: 0.2668 - val_loss: 1.8458 - val_accuracy: 0.2529
Epoch 5/10
14/14 [==============================] - 9s 672ms/step - loss: 1.8637 - accuracy: 0.2552 - val_loss: 1.8390 - val_accuracy: 0.2436
Epoch 6/10
14/14 [==============================] - 8s 561ms/step - loss: 1.8995 - accuracy: 0.2274 - val_loss: 1.9044 - val_accuracy: 0.1995
Epoch 7/10
14/14 [==============================] - 9s 682ms/step - loss: 1.8808 - accuracy: 0.2367 - val_loss: 1.8292 - val_accuracy: 0.2251
Epoch

In [8]:
import joblib

# Assuming 'vgg16_model' is your trained VGG16 model
model_filename = '/content/resnet.pkl'

# Pickle the model
joblib.dump(model, model_filename)


['/content/resnet.pkl']

#VGG16

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [ ]:
model2 = Sequential()
model2.add(base_model)
model2.add(Flatten())
model2.add(Dense(256, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(num_classes, activation='softmax'))  # Replace 'num_classes' with the number of classes in your dataset


In [ ]:
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
num_epochs = 10

# Train the model
model2.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)

Epoch 1/10
14/14 [==============================] - 15s 698ms/step - loss: 5.3498 - accuracy: 0.2088 - val_loss: 1.8438 - val_accuracy: 0.2831
Epoch 2/10
14/14 [==============================] - 11s 772ms/step - loss: 1.8724 - accuracy: 0.2738 - val_loss: 1.8323 - val_accuracy: 0.2831
Epoch 3/10
14/14 [==============================] - 10s 661ms/step - loss: 1.9049 - accuracy: 0.2645 - val_loss: 1.9312 - val_accuracy: 0.2831
Epoch 4/10
14/14 [==============================] - 11s 768ms/step - loss: 1.9342 - accuracy: 0.2367 - val_loss: 1.9223 - val_accuracy: 0.2831
Epoch 5/10
14/14 [==============================] - 10s 726ms/step - loss: 1.9222 - accuracy: 0.2784 - val_loss: 1.9081 - val_accuracy: 0.2831
Epoch 6/10
14/14 [==============================] - 11s 781ms/step - loss: 1.8986 - accuracy: 0.2877 - val_loss: 1.8921 - val_accuracy: 0.2831
Epoch 7/10
14/14 [==============================] - 10s 695ms/step - loss: 1.8841 - accuracy: 0.2831 - val_loss: 1.8770 - val_accuracy: 0.2831